PROWLER.io Coding Challenge
===

Description of the Disjoint Set

------

### Problem

There are `N` players on the field passing the ball to each other. Initially, the ball can be dropped to any player on the field. The player possessing the ball can throw it to another one _if and only if_ they see one another. The group of players which can pass the ball between themselfs form a team. There is a chance that the player, which got the ball in the beginning, doesn't see anybody and thereafter he makes its own 'lordly' team. The task is to find the team with largest number of members.

The players and their visibily are described in a file with following format:

- File contains `M` lines, where `M > 0` and can be less, equal or greater than `N`, number of players
- Each line is a separated by comma player names (or ids)
- First player name in the list belongs to the _key_ player. This is a player which potentially can have the ball and the rest of the list are other player which he can see, there is no guarantee that opposite is true.

In [13]:
# Players description file example

players = """George,Beth, Sue
             Rick,Anne
             Anne,  Beth
             Beth, Anne,George
             Sue, Beth"""

### Graph representation

Let's convert English description of the challenge to Computer Science terminology. The players are vertices in a graph. Visibility some of the players to one particular player can be expressed as outgoing directed edges. Taking into account these definitions of vertices and edges, we can build the graph using player description file. For e.g. we have an input file:

In [14]:
# Graph declaration and in the same time it is a players description file with wierd player's names

graph = """A,C,B
           B,A,D
           C,A,F
           D,C,B
           F,E
           E,F
           K,L"""

Then our graph looks like:

![Directed Graph](images/directed.png)

Once we build the directed graph, nothing prevents us from finding largest team using simple traversing algorithms, BFS or DFS, and marking visited vertices and cetera. Hold on! Wait! Is that really true?! Of course not. There is one important property that we have to remember: the ball can move from player to player _iif_ they see each other. In fact, the "visibility" property gives us a credit to boil down the _directed graph_ into _undirected graph_. The previous must be transformed into:

![Undirected Graph](images/undirected.png)

You may notice that now we observe distinct connected components. In fact, these components are player's teams. So, after this modification we are able to run traversing DFS or BFS on undirected graph, keeping list of visited vertices and stack of unexplored vertices, meanwhile counting members of a connected component. But, there is simpler way to accomplish the same task - disjoint set.

### Disjoint Set

Disjoint set is a data structure which speaks for itself. E.g. connected components of the undirected graph above can be seen as:

In [21]:
# str() conversiononly here is only for errors suppression

subset1 = str(['A', 'B', 'C', 'D'])
subset2 = str(['E', 'F'])
subset3 = str(['K'])
subset4 = str(['L'])
disjoint_set = set([subset1, subset2, subset3, subset4])
disjoint_set

{"['A', 'B', 'C', 'D']", "['E', 'F']", "['K']", "['L']"}

More interesting what is behind of this datas tructure, in other words how it can be implemented.

In conception of sets the connected components of the graph are not tied with each other with particular edges, so we can say that all items in the set are linked via invisible edges composing a clique. With relaxing connection property we got an opportunity to use another data structure like trees.

Each subset is represented as a tree with properties:

- All vertices in a tree are unique and not shared with other trees
- Root vertex has a self-link
- Root vertex acts as the index of the set
- Non-root vertices point to their parents
- Each tree has a rank metric - number of unions were made to construct the tree.
- Links have nothing to do with edges from source graph (It is not a property, just clarification)

The disjoint set datastructure has two major operations:

- Find set to which vertex belongs to
- Union of two sets

Let's take a look at an example. We have 6-element set. In disjoint set all of them are individual subsets and they are all roots. Initially, 1 size set has 0 rank.

![Initial State](images/1.png)

Then we explore that there are some undirected edges in the graph between 1 and 2. At first, we search roots of sets where 1 and 2 are located. Apparently, they are only vertices in their sets, and we can just union them building new set, with one condition: root of the set with larger rank becomes parent for smaller one. Both 1 and 2 roots have equal ranks, then new root is chosen randomly, in our case new root is 1:

![Initial State](images/2.png)

Same story happens with 3 and 4 vertices:

![Initial State](images/3.png)

It is time to merge sets larger than one. As you can see, we merged set 4 with 1. The 1 won the tournament for root's power again getting rank 2.

![Initial State](images/4.png)

New edge is appeared between 5 and 6. Their sets are merged into set with root 5. Rank 1 is assigned to the outcome set.

![Initial State](images/5.png)

Nothing interesting again?! Just merging sets with index 5 and 1. Powerful set 1 gets more points - rank 3. It is getting dangerous, isn't it? The depth of tree is growing. What if we had seen edge between 5th and 3rd vertices instead of 5th and 1st? We would get the same unstoppable set 1, but with depth 5. So, whenever there is a new vertex on horizon, we must to perform operation _find set_ and for now it will cost us `O(V)`, where `V` is a number of vertices. We can make it better.

![Initial State](images/6.png)

Now, we have an edge between 7th and 3rd vertices on our plate. Because 7 is new vertex for disjoint set, it is added and made as root for itself. Then, according to _adding an edge_ procedure in disjoint set, first we search for 7th and 3rd roots getting 7th and 1th vertices correspondingly. In latest example we have seen that _find set_ operation costs `O(V)`, here we will optimize it up to `O(a(V))`, where **`a(V)` is about 1**. During _find set_ operation we keep list of all visited nodes along path from requested vertex towards to the root. Here, path is `[3, 4]`.

![Initial State](images/7.png)

Once we get our path, we update parent links of path vertices to the root's value. Then _union_ operation links 7th vertex to 1th root, where 7th becomes its descendant. 1st set gets rank 4th:

![Initial State](images/8.png)

Because union operation entirely depends on _find set_ procedure, the cost for it also equals `O(a(n))`.

That's it. It is very simple data structure with good properties for solving this task.

### Examples

Consider previous graph as an example. The largest connected component should be equal 4.

In [10]:
import prowler

graph = """A,C,B
           B,A,D
           C,A,F
           D,C,B
           F,E
           E,F
           K,L"""

# Parse input players description
lines = graph.split("\n")
game = prowler.PassBallGame()
players = [game.parse_player_input(line) for line in lines]

# Add players to the game
_ = [game.add_player(*player) for player in players]

# Print largest team
print(game.largest_team)

4


### Major parts of ballgame code

1) Transformation from directed to undirected graph:

In [12]:
    def add_player(self, player, visible_players=[]):
        """
        Adds new player to the game.

        Args:
            player: Id of the player.
            visible_players: List of players which the player can see.
        """
        self._disjoint.add_vertex(player)

        for visible_player in visible_players:
            self._disjoint.add_vertex(visible_player)

            edge = (player, visible_player)
            self._see_each_other.add(edge)

            edge_reversed = edge[::-1]
            if edge_reversed in self._see_each_other:
                self._disjoint.add_edge(edge)

2) _Find set_ procedure in disjoint set data structure:

In [14]:
    def _find_root(self, vertex):
        """
        Finds the root of set to which vertex belongs to.

        Args:
            vertex:
        """
        if not self.exists(vertex):
            raise ValueError('Input vertex "{vertex}" does not exist'
                             .format(vertex=vertex))

        path = []
        current, parent = vertex, self._vertex_parent(vertex)
        while current != parent:
            path.append(current)
            current, parent = parent, self._vertex_parent(parent)

        # Path compression
        if len(path) > 1:
            for path_vertex in path:
                self._set_vertex_parent(path_vertex, parent)

        return parent

3) Union operation for two subsets:

In [15]:
    def _union(self, a_root, b_root):
        """
        Makes union of two sets specified by their roots.
        The combined set will have vertex with highest rank.
        The root of union gets rank equals `higest rank + 1`.
        If both vertices have equal ranks, then new root is
        chosen randomly.

        Args:
            a_root: Vertex value, which points to itself.
            b_root: Vertex value, which points to itself

        Raises:
            ValueError: Whenever vertices are not roots, i.e.
                        they don't have self-links.

        """

        err = 'Union for non root vertex "{vertex}" cannot be performed'

        if not self._is_root(a_root):
            raise ValueError(err.format(vertex=a_root))

        if not self._is_root(b_root):
            raise ValueError(err.format(vertex=b_root))

        if a_root == b_root:
            return

        a_rank = self._root_rank(a_root)
        b_rank = self._root_rank(b_root)

        # NOTE: Root with larger rank will set as root for new composed union.
        #       If ranks are equal, then `a_root` node becomes new root and
        #       `b_root` turns to the `a_root` descendant.
        if a_rank < b_rank:
            a_root, b_root = b_root, a_root

        self._set_root_rank(b_root, 0)
        self._set_vertex_parent(b_root, a_root)
        self._set_root_rank(a_root, a_rank + 1)
        self._steal_count(a_root, b_root)
        self._update_largest_set(a_root)

## Thank you for reading!